In [0]:
%sh 
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
display(dbutils.fs.ls("file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos"))

path,name,size,modificationTime
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/roses/,roses/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/sunflowers/,sunflowers/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/tulips/,tulips/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/daisy/,daisy/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/dandelion/,dandelion/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/sample/,sample/,4096,1722578435364
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/LICENSE.txt,LICENSE.txt,418049,1722553564654


In [0]:
# Define the base directory for images
img_dir = "file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos"

# Now proceed with the rest of your code in Cell 3
sample_img_dir = img_dir + "/sample"
dbutils.fs.mkdirs(sample_img_dir)
files = dbutils.fs.ls(img_dir + "/tulips")[0:1] + dbutils.fs.ls(img_dir + "/daisy")[0:2]
for f in files:
  dbutils.fs.cp(f.path, sample_img_dir)
display(dbutils.fs.ls(sample_img_dir))

path,name,size,modificationTime
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/sample/3498663243_42b39b4185_m.jpg,3498663243_42b39b4185_m.jpg,23948,1722578451842
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/sample/2001380507_19488ff96a_n.jpg,2001380507_19488ff96a_n.jpg,38478,1722578453489
file:/Workspace/Users/adagniew407@student.sfbu.edu/flower_photos/sample/14272874304_47c0a46f5a.jpg,14272874304_47c0a46f5a.jpg,119329,1722578455156


In [0]:
# Use the binary file data source for loading image data
image_df = spark.read.format("image").load(sample_img_dir)
image_df.show()

+--------------------+
|               image|
+--------------------+
|{file:/Workspace/...|
|{file:/Workspace/...|
|{file:/Workspace/...|
+--------------------+



In [0]:
display(image_df)

In [0]:
from pyspark.sql.functions import lit

# Assuming img_dir is defined and points to the base directory containing 'tulips' and 'daisy' subdirectories
tulips_df = spark.read.format("image").load(img_dir + "/tulips").withColumn("label", lit(1))
daisy_df = spark.read.format("image").load(img_dir + "/daisy").withColumn("label", lit(0))

# Splitting the data into training and test sets
tulips_train, tulips_test = tulips_df.randomSplit([0.1, 0.9])  # Adjusted for simplicity
daisy_train, daisy_test = daisy_df.randomSplit([0.1, 0.9])     # Adjusted for simplicity

# Combining the training and test sets
train_df = tulips_train.unionByName(daisy_train)
test_df = tulips_test.unionByName(daisy_test)

# Repartitioning to manage memory usage more efficiently
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

p_model = p.fit(train_df)

In [0]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# Convert features array to VectorUDT
def array_to_vector(array):
    return Vectors.dense(array)

array_to_vector_udf = udf(array_to_vector, VectorUDT())

# Apply the conversion to create the 'features_vector' column
train_df = train_df.withColumn("features_vector", array_to_vector_udf(col("features")))

# Check for the existence of 'features_vec' column and drop it if it exists
if 'features_vec' in train_df.columns:
    train_df = train_df.drop('features_vec')

# Assemble features into a single vector column
assembler = VectorAssembler(inputCols=["features_vector"], outputCol="features_vec")

# Transform the DataFrame to add the features_vec column
train_df = assembler.transform(train_df)

# Define and train the logistic regression model
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[assembler, lr])

# Fit the pipeline model
p_model = p.fit(train_df)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
def _p1(v):
  return float(v.array[1])
p1 = udf(_p1, DoubleType())

df = tested_df.withColumn("p_1", p1(tested_df.probability))
wrong_df = df.orderBy(expr("abs(p_1 - label)"), ascending=False)
display(wrong_df.select("filePath", "p_1", "label").limit(10))

In [0]:
from sparkdl import readImages, DeepImagePredictor

image_df = readImages(sample_img_dir)

predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="InceptionV3", decodePredictions=True, topK=10)
predictions_df = predictor.transform(image_df)

display(predictions_df.select("filePath", "predicted_labels"))

In [0]:
df = p_model.transform(image_df)
display(df.select("filePath", (1-p1(df.probability)).alias("p_daisy")))

In [0]:
from sparkdl import readImages, TFImageTransformer
from sparkdl.transformers import utils
import tensorflow as tf

image_df = readImages(sample_img_dir)

g = tf.Graph()
with g.as_default():
    image_arr = utils.imageInputPlaceholder()
    resized_images = tf.image.resize_images(image_arr, (299, 299))
    # the following step is not necessary for this graph, but can be for graphs with variables, etc
    frozen_graph = utils.stripAndFreezeGraph(g.as_graph_def(add_shapes=True), tf.Session(graph=g), [resized_images])
      
transformer = TFImageTransformer(inputCol="image", outputCol="transformed_img", graph=frozen_graph,
                                 inputTensor=image_arr, outputTensor=resized_images,
                                 outputMode="image")
tf_trans_df = transformer.transform(image_df)

In [0]:
from keras.applications import InceptionV3

model = InceptionV3(weights="imagenet")
model.save('/tmp/model-full.h5')  # saves to the local filesystem
# move to a permanent place for future use
dbfs_model_path = 'dbfs:/models/model-full.h5'
dbutils.fs.cp('file:/tmp/model-full.h5', dbfs_model_path)  

In [0]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

def loadAndPreprocessKerasInceptionV3(uri):
  # this is a typical way to load and prep images in keras
  image = img_to_array(load_img(uri, target_size=(299, 299)))  # image dimensions for InceptionV3
  image = np.expand_dims(image, axis=0)
  return preprocess_input(image)

dbutils.fs.cp(dbfs_model_path, 'file:/tmp/model-full-tmp.h5')
transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile='/tmp/model-full-tmp.h5',  # local file path for model
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

files = ["/dbfs" + str(f.path)[5:] for f in dbutils.fs.ls(sample_img_dir)]  # make "local" file paths for images
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("uri")

keras_pred_df = transformer.transform(uri_df)

In [0]:
display(keras_pred_df.select("uri", "predictions"))

In [0]:
dbutils.fs.rm(img_dir, recurse=True)
dbutils.fs.rm(dbfs_model_path)